In [28]:
from statsmodels.regression.rolling import RollingOLS 
import  pandas as pd
import  numpy as np
import  matplotlib.pyplot as plt
import  statsmodels.api as sm
import  pandas_datareader.data as web
import  datetime as dt
import  yfinance as yf
import  pandas_ta
import  warnings
warnings.filterwarnings("ignore")

In [29]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500[0]['Symbol'] = sp500[0]['Symbol'].str.replace('.', '-')
sp500_list = sp500[0]['Symbol'].unique().tolist()

end_date = '2023-01-22'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)
df = yf.download(tickers=sp500_list, start=start_date, end=end_date).stack()
print(df.head(5))

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1422075600, endDate = 1674363600")


Price              Adj Close      Close       High        Low       Open  \
Date       Ticker                                                          
2015-01-26 A       36.304672  39.150002  39.169998  38.470001  38.790001   
           AAL     52.539070  55.450001  56.200001  54.730000  55.889999   
           AAPL    25.277872  28.275000  28.590000  28.200001  28.434999   
           ABBV    42.971256  62.830002  63.040001  61.680000  62.259998   
           ABT     37.234192  44.150002  44.200001  43.419998  43.740002   

Price                   Volume  
Date       Ticker               
2015-01-26 A         1510900.0  
           AAL      13353900.0  
           AAPL    222460000.0  
           ABBV      8181100.0  
           ABT       4376200.0  


In [30]:
df.columns = ['adj close', 'close', 'high', 'low', 'open', 'volume']  
df.index.names = ['date', 'ticker']
df.head(5)

adj close      close       high        low       open  \
date       ticker                                                          
2015-01-26 A       36.304672  39.150002  39.169998  38.470001  38.790001   
           AAL     52.539070  55.450001  56.200001  54.730000  55.889999   
           AAPL    25.277872  28.275000  28.590000  28.200001  28.434999   
           ABBV    42.971256  62.830002  63.040001  61.680000  62.259998   
           ABT     37.234192  44.150002  44.200001  43.419998  43.740002   

                        volume  
date       ticker               
2015-01-26 A         1510900.0  
           AAL      13353900.0  
           AAPL    222460000.0  
           ABBV      8181100.0  
           ABT       4376200.0

In [31]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])


In [32]:
df['close'].dropna()

date        ticker
2015-01-26  A          39.150002
            AAL        55.450001
            AAPL       28.275000
            ABBV       62.830002
            ABT        44.150002
                         ...    
2023-01-20  YUM       126.629997
            ZBH       124.089996
            ZBRA      297.029999
            ZION       51.480000
            ZTS       163.809998
Name: close, Length: 989448, dtype: float64

In [33]:
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd_result = pandas_ta.macd(close=close, length=20)
    
    if macd_result is not None:
        macd = macd_result.iloc[:, 0]
        return macd.sub(macd.mean()).div(macd.std())
    else:
        # Handle the case where macd_result is None
        return None

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

mean_volume = np.mean(df['volume'])
df['dollar_volume'] = (df['close'] * df['volume'] / mean_volume).astype(int)


In [34]:
df['dollar_volume']

date        ticker
2015-01-26  A           11
            AAL        142
            AAPL      1211
            ABBV        99
            ABT         37
                      ... 
2023-01-20  YUM         40
            ZBH         41
            ZBRA        17
            ZION        24
            ZTS        153
Name: dollar_volume, Length: 989448, dtype: int32

# 3 Aggregate to monthly level and filter top 150 most liquid stocks each month

In [35]:
last_cols = [c for c in df.columns.unique(0) if c not in ['open', 'dollar_volume', 'close', 'high', 'open', 'low']]
data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
            df.unstack()[last_cols].resample('M').last().stack('ticker')],
         axis=1)).dropna()

In [36]:
data.head

<bound method NDFrame.head of                    dollar_volume   adj close       volume  garman_klass_vol  \
date       ticker                                                             
2015-03-31 A           15.863636   38.625145    1659200.0         -0.001572   
           AAL        184.136364   50.110554    8386400.0         -0.001614   
           AAPL      1254.863636   27.919876  168362400.0         -0.005545   
           ABBV       148.545455   40.037201   12313600.0         -0.052936   
           ABT         46.181818   39.072712    5358100.0         -0.012572   
...                          ...         ...          ...               ...   
2023-01-31 CTVA        31.307692   61.447720    2570800.0          0.000140   
           CARR        31.769231   41.909531    5217500.0          0.000213   
           OTIS        28.307692   79.263443    1787600.0          0.000184   
           ABNB        96.307692  101.279999    5104000.0          0.000335   
           CEG        

* Calculate a 5-year rolling averageof dollar volume for each stock before filtering

In [37]:
data['dollar_volume'] = (data['dollar_volume'].unstack('ticker').rolling(5*12).mean()).stack()
data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

In [38]:
data

adj close       volume  garman_klass_vol        rsi  \
date       ticker                                                         
2020-02-29 AAL      19.049999   37093200.0          0.002690  25.579469   
           AAPL     66.651978  426510000.0          0.002909  34.635062   
           ABBV     71.975410   18450000.0         -0.006350  43.432026   
           ABT      71.965286   16671900.0         -0.000368  26.562275   
           ACN     170.531387    5785200.0         -0.000218  24.475471   
...                       ...          ...               ...        ...   
2023-01-31 WMT      46.142857   29543100.0          0.000156  43.580085   
           WYNN     98.263878    1967300.0          0.000079  68.658347   
           XOM     108.598480   16515600.0          0.000076  57.754169   
           ETSY    131.820007    2475500.0          0.000550  55.777476   
           PYPL     79.089996   12193200.0          0.000579  54.867175   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2020-02-29 AAL     3.079842  3.325488  3.571133  0.842555 -1.821580  
           AAPL    4.232049  4.340146  4.448242  0.516650 -1.099161  
           ABBV    4.234344  4.342609  4.450873  0.622132 -0.127989  
           ABT     4.317090  4.404311  4.491531  0.528605 -1.978772  
           ACN     5.185128  5.280504  5.375881  0.464353 -1.443867  
...                     ...       ...       ...       ...       ...  
2023-01-31 WMT     3.846316  3.873340  3.900363  0.606132 -0.873279  
           WYNN    4.342848  4.506016  4.669184 -1.006656  1.471120  
           XOM     4.626746  4.666128  4.705510  1.657263  0.920401  
           ETSY    4.741524  4.841865  4.942207  0.516154  0.435784  
           PYPL    4.210163  4.326182  4.442200 -0.328513  0.241212  

[5364 rows x 9 columns]